### 合并文件

In [ ]:
%run ../process_data.py meta_Grocery.json reviews_Grocery_5.json #暂不执行split_test()---20180118/20190119

### 生成用户序列

In [ ]:
import sys
import hashlib
import random

fin = open("jointed-new", "r")
fo = open("jointed-new-userseq", "w")

last_user = "0"
common_fea = ""
line_idx = 0
for line in fin:
    items = line.strip().split("\t")
    clk = int(items[0])
    user = items[1]
    movie_id = items[2]
    dt = items[4]
    cat1 = items[5]

    if user != last_user:
        movie_id_list = []
        cate1_list = []
        #print >> fo, items[1] + "\t" + user + "\t" + movie_id + "\t" + cat1 +"\t" + "" + "\t" + "" 
    else:
        history_clk_num = len(movie_id_list)
        cat_str = ""
        mid_str = ""
        for c1 in cate1_list:
            cat_str += c1 + ""
        for mid in movie_id_list:
            mid_str += mid + ""
        if len(cat_str) > 0: cat_str = cat_str[:-1]
        if len(mid_str) > 0: mid_str = mid_str[:-1]
        if history_clk_num >= 1:    # 8 is the average length of user behavior
            print(items[0] + "\t" + user + "\t" + movie_id + "\t" + cat1 +"\t" + mid_str + "\t" + cat_str,file=fo)
    last_user = user
    if clk:
        movie_id_list.append(movie_id)
        cate1_list.append(cat1)                
    line_idx += 1

### 生成物品序列

In [ ]:
import sys
import hashlib
import random

fin = open("jointed-new-userseq", "r")
ftest = open("jointed-new-userseq-itemseq", "w")##

item_map = {}##key=item_id(clk=1),value = ()
last_user = "0"
common_fea = ""
line_idx = 0
for line in fin:
    items = line.strip().split("\t")
    clk = int(items[0])
    user = items[1]
    item_id = items[2]
    cat1 = items[3]
    mid_list = items[4]
    cat_list = items[5]
    
    if clk==1:
        if item_id not in item_map:
            item_map[item_id]= []
        item_map[item_id].append((user,mid_list,cat_list,len(mid_list.split(""))))
        
    line_idx += 1
    
sorted_item_map = {}
for key in item_map:
    sorted_item_map[key] = sorted(item_map[key], key=lambda x:x[3], reverse=True)#按照长度排序

fin = open("jointed-new-userseq", "r")    
i = 0    
for line in fin:
    items = line.strip().split("\t")
    clk = int(items[0])
    user = items[1]
    item_id = items[2]
    cat1 = items[3]
    mid_list = items[4]
    cat_list = items[5]
    
    history_users_feats = ""
    history_users_num = 0
    if item_id in sorted_item_map:
        temp = sorted_item_map[item_id]
        for t in temp:
            item_bhvs_feat=t[0]+'_'+t[1]+'_'+t[2]
            history_users_feats = history_users_feats+";"+item_bhvs_feat
            history_users_num+=1
            
    print(items[0] + "\t" + user + "\t" + item_id + "\t" + cat1 +"\t" + mid_list + "\t" + cat_list + "\t" + history_users_feats + "\t" + str(history_users_num),file=ftest)
    i+=1

### 生成voc

In [ ]:
import pickle
import random

f_train = open("jointed-new-userseq-itemseq", "r")#根据"local_train_splitByUser"和"local_test_splitByUser"生成
uid_dict = {}
mid_dict = {}
cat_dict = {}

iddd = 0
for line in f_train:
    arr = line.strip("\n").split("\t")
    clk = arr[0]
    uid = arr[1]
    mid = arr[2]
    cat = arr[3]
    mid_list = arr[4]
    cat_list = arr[5]
    
    history_users_feats = arr[6]
    history_users_num = arr[7]
    
    if uid not in uid_dict:
        uid_dict[uid] = 0
    uid_dict[uid] += 1
    if mid not in mid_dict:
        mid_dict[mid] = 0
    mid_dict[mid] += 1
    if cat not in cat_dict:
        cat_dict[cat] = 0
    cat_dict[cat] += 1
    
    if len(mid_list) == 0:
        continue
    for m in mid_list.split(""):
        if m not in mid_dict:
            mid_dict[m] = 0
        mid_dict[m] += 1
    for c in cat_list.split(""):
        if c not in cat_dict:
            cat_dict[c] = 0
        cat_dict[c] += 1
    
    if int(history_users_num) == 0:
        continue
    for bhvs in history_users_feats.split(";")[1:]:
        arr = bhvs.split("_")
        if arr[1].strip() == "":
            continue
        bhv_uid = arr[0]
        bhv_mid_list = arr[1]
        bhv_cat_list = arr[2]

        if bhv_uid not in uid_dict:
            uid_dict[bhv_uid] = 0
        uid_dict[bhv_uid] += 1

        for bm in bhv_mid_list.split(""):
            if bm not in mid_dict:
                mid_dict[bm] = 0
            mid_dict[bm] += 1
        for bc in bhv_cat_list.split(""):
            if bc not in cat_dict:
                cat_dict[bc] = 0
            cat_dict[bc] += 1        
    iddd+=1

#sorted_uid_dict = list(uid_dict.keys())
#random.shuffle(sorted_uid_dict)
sorted_uid_dict = sorted(uid_dict.items(), key=lambda x:x[1], reverse=True)
sorted_mid_dict = sorted(mid_dict.items(), key=lambda x:x[1], reverse=True)
sorted_cat_dict = sorted(cat_dict.items(), key=lambda x:x[1], reverse=True)

uid_voc = {}
index = 0
for key, value in sorted_uid_dict:
    uid_voc[key] = index
    index += 1

mid_voc = {}
mid_voc["default_mid"] = 1
index = 2
for key, value in sorted_mid_dict:
    mid_voc[key] = index
    index += 1

cat_voc = {}
cat_voc["default_cat"] = 1
index = 2
for key, value in sorted_cat_dict:
    cat_voc[key] = index
    index += 1

pickle.dump(uid_voc, open("uid_voc.pkl", "wb"))
pickle.dump(mid_voc, open("mid_voc.pkl", "wb"))
pickle.dump(cat_voc, open("cat_voc.pkl", "wb"))

### 生成负userseq并符合模型输入要求

In [ ]:
import pickle
import numpy
import json
import random

import gzip

def load_dict(filename):
    F=open(filename,'rb')
    return pickle.load(F)


def fopen(filename, mode='r'):
    if filename.endswith('.gz'):
        return gzip.open(filename, mode)
    return open(filename, mode)

def generator_neg(source,
                 uid_voc,
                 mid_voc,
                 cat_voc,
                 maxlen=None,
                 skip_empty=False,
                 shuffle_each_epoch=False,
                 sort_by_length=True,
                 minlen=None):
    if shuffle_each_epoch:
        source_orig = source
        source = random.shuffle.main(source_orig, temporary=True)
    else:
        source = fopen(source, 'r')#数据文件
           
    source_dicts = []
    for source_dict in [uid_voc, mid_voc, cat_voc]:
        source_dicts.append(load_dict(source_dict))#文件
        

    f_meta = open("item-info", "r")
    meta_map = {}#asin-category
    for line in f_meta:
        arr = line.strip().split("\t")
        if arr[0] not in meta_map:
            meta_map[arr[0]] = arr[1]
                
    meta_id_map ={}#asin-category
    for key in meta_map:#asin
        val = meta_map[key]#category
        if key in source_dicts[1]:#mid_voc:asin-index
             mid_idx = source_dicts[1][key]#asin_index
        else:
            mid_idx = 0
        if val in source_dicts[2]:#cat_voc
            cat_idx = source_dicts[2][val]#category_index
        else:
            cat_idx = 0
        meta_id_map[mid_idx] = cat_idx#asin_index----category_index

    f_review = open("reviews-info", "r")
    mid_list_for_random = []
    for line in f_review:
        arr = line.strip().split("\t")#userId,asin,rating,time
        tmp_idx = 0
        if arr[1] in source_dicts[1]:#mid_voc:asin-index
            tmp_idx = source_dicts[1][arr[1]]
        mid_list_for_random.append(tmp_idx)

    n_uid = len(source_dicts[0])
    n_mid = len(source_dicts[1])
    n_cat = len(source_dicts[2])

    shuffle = shuffle_each_epoch

    end_of_data = False
    
    feats = []
    target = []
    
    num=0
    for line in source:
        ss = line.strip("\n").split("\t")#strip() 方法用于移除字符串头尾指定的字符（默认为空格）或字符序列

        if len(ss)==0:
            continue
        # sort by  history behavior length

        uid = source_dicts[0][ss[1]] if ss[1] in source_dicts[0] else 0
        mid = source_dicts[1][ss[2]] if ss[2] in source_dicts[1] else 0
        cat = source_dicts[2][ss[3]] if ss[3] in source_dicts[2] else 0

        tmp = []
        for fea in ss[4].split(""):
            m = source_dicts[1][fea] if fea in source_dicts[1] else 0
            tmp.append(m)
        mid_list = tmp

        tmp1 = []
        for fea in ss[5].split(""):
            c = source_dicts[2][fea] if fea in source_dicts[2] else 0
            tmp1.append(c)
        cat_list = tmp1

        lens = len(mid_list)
        # read from source file and map to word index

        #if len(mid_list) > self.maxlen:
        #    continue
        if lens==0:#如果用户序列为0，直接跳过
            continue
        if minlen != None:
            if len(mid_list) <= minlen:
                continue
        if skip_empty and (not mid_list):
            continue

        #user behavior负样本，序列中的每个行为对应生成一个负样本
        noclk_mid_list = []
        noclk_cat_list = []
        for pos_mid in mid_list:
            while True:
                noclk_mid_indx = random.randint(0, len(mid_list_for_random)-1)
                noclk_mid = mid_list_for_random[noclk_mid_indx]
                if noclk_mid != pos_mid:
                    noclk_mid_list.append(noclk_mid)
                    noclk_cat_list.append(meta_id_map[noclk_mid])
                    break

        #item behavior 用户id_lens_items_cats
        if int(ss[7]) == 0:
            continue

        item_bhvs_uid_feats = []
        item_bhvs_num_feats = []
        item_bhvs_id_feats = []
        item_bhvs_cat_feats = []
        for bhvs in ss[6].split(";")[1:]:
            arr = bhvs.split("_")
            if arr[1].strip() == "":
                continue

            bhv_uid = source_dicts[0][arr[0]] if arr[0] in source_dicts[0] else 0
            item_bhvs_uid_feats.append(bhv_uid)

            id_tmp_list = []
            for fea in arr[1].split(""):
                m = source_dicts[1][fea] if fea in source_dicts[1] else 0
                id_tmp_list.append(m)
            item_bhvs_id_feats.append(id_tmp_list)

            item_bhvs_num_feats.append(len(id_tmp_list))

            cat_tmp_list = []
            for fea in arr[2].split(""):
                c = source_dicts[2][fea] if fea in source_dicts[2] else 0
                cat_tmp_list.append(c)
            item_bhvs_cat_feats.append(cat_tmp_list)

        feats.append([uid, mid, cat, lens, mid_list, cat_list, noclk_mid_list, noclk_cat_list, item_bhvs_uid_feats,item_bhvs_num_feats,item_bhvs_id_feats,item_bhvs_cat_feats,int(ss[7])])
        target.append([float(ss[0]), 1-float(ss[0])])
        num+=1
    print("sample counts",num) 
    return feats,target

In [ ]:
fin = "jointed-new-userseq-itemseq"
uid_voc = "uid_voc.pkl"
mid_voc = "mid_voc.pkl"
cat_voc = "cat_voc.pkl"

In [ ]:
feats,target = generator_neg(fin, uid_voc, mid_voc, cat_voc, shuffle_each_epoch=False)

In [ ]:
import pandas as pd
def getlist(features,labels):
    data = []
    for s,t in zip(features,labels):
        uid = s[0]
        mid = s[1]
        cat = s[2]
        histlen = s[3]
        mid_list = s[4]
        cat_list = s[5]
        mid_neg_list = s[6]
        cat_neg_list = s[7]
        
        item_uid_list = s[8]
        item_midlen_list = s[9]
        item_mid_list = s[10]
        item_cat_list = s[11]
        item_histlen = s[12]

        label = int(t[0])
        data.append((uid, mid, cat, histlen, mid_list, cat_list, mid_neg_list, cat_neg_list, item_uid_list, item_midlen_list,\
                     item_mid_list, item_cat_list, item_histlen, label))
    return data

In [ ]:
datalist = getlist(feats,target)

In [ ]:
random.shuffle(datalist)

In [ ]:
len(datalist)

##### 按照8:2划分数据集

In [ ]:
cut_num =int(len(datalist)*0.8)

In [ ]:
trdatalist=datalist[:cut_num]

In [ ]:
tedatalist=datalist[cut_num:]

In [ ]:
random.shuffle(trdatalist)
random.shuffle(tedatalist)

### 保存

In [ ]:
f1=open("uid_voc.pkl",'rb')
f2=open("mid_voc.pkl",'rb')
f3=open("cat_voc.pkl",'rb')
user = pickle.load(f1)
item = pickle.load(f2)
cate = pickle.load(f3)

In [ ]:
user_count, item_count, cate_count=\
    len(user), len(item), len(cate)
print('user_count: %d\titem_count: %d\tcate_count: %d' %
      (user_count, item_count, cate_count))

In [ ]:
#### 第一次保存完整的处理后的数据结果

In [ ]:
import pickle
with open('Grocery1.pkl', 'wb') as f:
    pickle.dump(trdatalist, f, pickle.HIGHEST_PROTOCOL)#整型，最高协议版本
    pickle.dump(tedatalist, f, pickle.HIGHEST_PROTOCOL)#整型，最高协议版本
    pickle.dump((user_count, item_count, cate_count), f, pickle.HIGHEST_PROTOCOL)

#### 删除相似用户中的用户本身

In [ ]:
with open('Grocery1.pkl', 'rb') as f: 
    train_data = pickle.load(f)
    test_data = pickle.load(f)
    user_count, item_count, cate_count = pickle.load(f)

In [ ]:
import copy
trdata = copy.deepcopy(train_data)
tedata = copy.deepcopy(test_data)

In [ ]:
#(uid, mid, cat, histlen, mid_list, cat_list, mid_neg_list, cat_neg_list, item_uid_list, item_midlen_list,\
                     #item_mid_list, item_cat_list, item_histlen, label)

In [ ]:
def cutself(data):
    #random.shuffle(data)
    ts=data
    counts = len(ts)
    
    datacself = []
    for s in ts:
        uid = s[0]
        mid = s[1]
        cat = s[2]
        histlen = s[3]
        mid_list = s[4]
        cat_list = s[5]
        mid_neg_list = s[6]
        cat_neg_list = s[7]
        
        item_uid_list = s[8]
        item_midlen_list = s[9]
        item_mid_list = s[10]
        item_cat_list = s[11]
        item_histlen = s[12]
        label = s[13]
        
        if uid in item_uid_list:
            index = item_uid_list.index(uid)
            item_uid_list.pop(index)
            item_midlen_list.pop(index)
            item_mid_list.pop(index)
            item_cat_list.pop(index)
            #item_histlen -= 1
            
        if len(item_uid_list)==0:
            continue
            
        datacself.append((uid, mid, cat, histlen, mid_list, cat_list, mid_neg_list, cat_neg_list, item_uid_list, item_midlen_list,\
                     item_mid_list, item_cat_list, len(item_uid_list), label))
    
    return datacself

In [ ]:
trdata_cs = cutself(trdata)
tedata_cs = cutself(tedata)

In [ ]:
with open('Grocery2.pkl', 'wb') as f:
    pickle.dump(trdata_cs, f, pickle.HIGHEST_PROTOCOL)#整型，最高协议版本
    pickle.dump(tedata_cs, f, pickle.HIGHEST_PROTOCOL)#整型，最高协议版本
    pickle.dump((user_count, item_count, cate_count), f, pickle.HIGHEST_PROTOCOL)

#### 细节处理
    For these three datasets, we set the maximum sequence length to 60 and the maximum number of similar users in a group to 50. 

In [ ]:
with open('Grocery2.pkl', 'rb') as f: 
    train_data = pickle.load(f)
    test_data = pickle.load(f)
    user_count, item_count, cate_count = pickle.load(f)

In [ ]:
def getdata_fixedlen(data):
    #random.shuffle(data)
    ts=data
    counts = len(ts)
    
    datacself = []
    for s in ts:
        uid = s[0]
        mid = s[1]
        cat = s[2]
        
        mid_list = s[4][-60:]
        cat_list = s[5][-60:]
        mid_neg_list = s[6][-60:]
        cat_neg_list = s[7][-60:]
        histlen = len(mid_list)
        
        item_uid_list = s[8][:50]
        item_midlen_list = []
        
        item_mid_list = []
        for fea in s[10]:
            a = fea[-60:]
            item_mid_list.append(a)
            item_midlen_list.append(len(a))
        item_mid_list=item_mid_list[:50]
        item_midlen_list=item_midlen_list[:50]
        
        item_cat_list = []
        for fea in s[11]:
            a = fea[-60:]
            item_cat_list.append(a)
        item_cat_list=item_cat_list[:50]
        
        #item_histlen = len(item_mid_list)
        label = s[13]
            
        datacself.append((uid, mid, cat, histlen, mid_list, cat_list, mid_neg_list, cat_neg_list, item_uid_list, item_midlen_list,\
                     item_mid_list, item_cat_list, len(item_uid_list), label))
    
    return datacself

In [ ]:
train_re = getdata_fixedlen(train_data)
test_re = getdata_fixedlen(test_data)

In [ ]:
import pickle
with open('Groceryf.pkl', 'wb') as f:#60,50
    pickle.dump(train_re, f, pickle.HIGHEST_PROTOCOL)#整型，最高协议版本
    pickle.dump(test_re, f, pickle.HIGHEST_PROTOCOL)#整型，最高协议版本
    pickle.dump((user_count, item_count, cate_count), f, pickle.HIGHEST_PROTOCOL)